In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
import math
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Guille\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guille\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
# Cargar un modelo de incrustaciones de palabras pre-entrenado (Word2Vec en este caso)
word2vec_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [6]:
df = pd.read_parquet('Rest_google.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'Rest_google.parquet'

In [14]:
def preprocess_input(input_text):
    # Tokenización y limpieza de texto
    tokens = word_tokenize(input_text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

In [21]:
def calculate_distance(user_location, restaurant_location):
    # Extraer las coordenadas de latitud y longitud del usuario y del restaurante
    user_lat, user_lon = user_location
    restaurant_lat, restaurant_lon = restaurant_location

    # Radio de la Tierra en kilómetros
    R = 6371.0

    # Convertir latitud y longitud de grados a radianes
    user_lat = math.radians(user_lat)
    user_lon = math.radians(user_lon)
    restaurant_lat = math.radians(restaurant_lat)
    restaurant_lon = math.radians(restaurant_lon)

    # Diferencia en latitud y longitud
    dlon = restaurant_lon - user_lon
    dlat = restaurant_lat - user_lat

    # Fórmula de Haversine para calcular la distancia
    a = math.sin(dlat / 2)**2 + math.cos(user_lat) * math.cos(restaurant_lat) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distancia en kilómetros
    distance = R * c

    return distance

In [22]:
def recommend_restaurants(input_text, df, word2vec_model, user_location):
    input_keywords = preprocess_input(input_text)

    # Obtener representaciones vectoriales para las palabras clave
    keyword_vectors = [word2vec_model[word] for word in input_keywords if word in word2vec_model.vocab]

    # Calcular la media de los vectores de palabras clave
    if keyword_vectors:
        input_vector = np.mean(keyword_vectors, axis=0)
        # Calcular la similitud de coseno entre el input y las categorías de los restaurantes
        df['similarity'] = df['category'].apply(lambda x: cosine_similarity([input_vector], [np.mean([word2vec_model[word] for word in x if word in word2vec_model.vocab], axis=0)])[0][0])

        # Calcular la distancia desde la ubicación del usuario
        df['distance'] = df['coord'].apply(lambda x: calculate_distance(user_location, x))

        # Ponderar similitud y distancia para obtener una puntuación final
        df['score'] = df['similarity'] + 0.5 * (1 - df['distance'])

        # Ordenar por puntuación
        recommended_restaurants = df.sort_values(by='score', ascending=False).head(5)
        return recommended_restaurants[['nombre_restaurante', 'categorias', 'ubicacion']]
    else:
        return "No se encontraron palabras clave válidas en el input."

In [23]:
input_text = "I want to eat sushi and pasta"
user_location = 'UserLocation'  # Reemplaza con la ubicación real del usuario
recommended_restaurants = recommend_restaurants(input_text, df, word2vec_model, (44.3069541, -89.8457834))
print(recommended_restaurants)

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4